In [ ]:
import glob
import io
import os
import re

In [ ]:
import keras
import numpy as np
import pylab as pl
import tensorflow as tf
from PIL import Image, ImageOps
from keras.applications import DenseNet121, ResNet50
from keras.applications.densenet import decode_predictions, preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization, Dense, Dropout, Flatten, ReLU
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from matplotlib import pyplot as plt
from numpy import argmax
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit, train_test_split

In [ ]:
#Prints the count of all images.
parent_dir = '/Users/Vicente/Downloads/ann/work/Arch/arch_img/'
count = 0
for subdir, dirs, files in os.walk(parent_dir):
    for file in files:
        im = str.lower(file)
        if im.endswith('.jpg'):
            count++
print(count)

In [ ]:
# DO NOT RUN (One time process)
# Mirrors all images
parent_dir = '/Users/Vicente/Downloads/ann/work/Arch/arch_img/'
count = 0
for subdir, dirs, files in os.walk(parent_dir):
    for file in files:
        im = str.lower(file)
        if im.endswith('.jpg'):
            temp = os.path.join(parent_dir, subdir)
            image_path = (os.path.join(temp, im))
            im = Image.open(image_path)
            imf = ImageOps.mirror(im)
            nojpg = image_path[:-4]
            new_path = (nojpg + "_mirror.jpg")
            print(new_path)
            imf.save(new_path, "JPEG")

In [ ]:
#Initizalizes the Densenet model and creates the classifier on top.
model = DenseNet121(weights='imagenet', include_top=False,
                    input_shape=[128, 128, 3])
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-100:]:
    layer.trainable = True
flat1 = Flatten()(model.layers[-1].output)
D1 = Dense(32, activation='relu')(flat1)
B3 = BatchNormalization()(D1)
R2 = ReLU()(B3)
Dr2 = Dropout(0.5)(R2)
D3 = Dense(32, activation='relu')(Dr2)
B2 = BatchNormalization()(D3)
output = Dense(25, activation='softmax')(B2)
model = Model(inputs=model.inputs, outputs=output)
model.summary()

In [ ]:
parent_dir = '/Users/Vicente/Downloads/ann/work/Arch/arch_img/'
for subdir, dirs, files in os.walk(parent_dir):
    print(subdir)

In [ ]:
catagories = {'Palladian architecture': 0,
              'Novelty architecture': 1,
              'International style': 2,
              'Chicago school architecture': 3,
              'Beaux-Arts architecture': 4,
              'Ancient Egyptian architecture': 5,
              'Gothic architecture': 6,
              'Tudor Revival architecture': 7,
              'Romanesque architecture': 8,
              'Colonial architecture': 9,
              'American craftsman style': 10,
              'Greek Revival architecture': 11,
              'Queen Anne architecture': 12,
              'Baroque architecture': 13,
              'Edwardian architecture': 14,
              'Art Nouveau architecture': 15,
              'Deconstructivism': 16,
              'Bauhaus architecture': 17,
              'Georgian architecture': 18,
              'Byzantine architecture': 19,
              'Postmodern architecture': 20,
              'Achaemenid architecture': 21,
              'Art Deco architecture': 22,
              'American Foursquare architecture': 23,
              'Russian Revival architecture': 24}
print(type(catagories))

In [ ]:
# creates a list of all image paths in parent dir
all_img_paths = glob.glob(os.path.join(parent_dir, '*.jpg'))
parent_dir = '/Users/Vicente/Downloads/ann/work/Arch/arch_img/'
for subdir, dirs, files in os.walk(parent_dir):
    for file in files:
        im = str.lower(file)
        if im.endswith('.jpg'):
            temp = os.path.join(parent_dir, subdir)
            image_path = (os.path.join(temp, im))
            all_img_paths.append(image_path)
print(len(all_img_paths))
print(all_img_paths)

In [ ]:
# creates a matrix with all the processed images ready to be input into the model.

IMG_SIZE_X = 128
IMG_SIZE_Y = 128
IMAGE_SIZE = 128
IMAGE_CHANNELS = 3

all_img_paths = glob.glob(os.path.join(parent_dir, '*.jpg'))
parent_dir = '/Users/Vicente/Downloads/ann/work/Arch/arch_img/'
for subdir, dirs, files in os.walk(parent_dir):
    for file in files:
        im = str.lower(file)
        if im.endswith('.jpg'):
            temp = os.path.join(parent_dir, subdir)
            image_path = (os.path.join(temp, im))
            all_img_paths.append(image_path)

sample_img_paths = all_img_paths[0:5000]

# np.random.shuffle(all_img_paths)
final = []
count = 0
# SWITCH TO ALL IMAGES WHEN READY
# could use preprocess_img to not have to use list
for img_path in all_img_paths:
    print(img_path)
    image = Image.open(img_path).resize(
        (IMAGE_SIZE, IMAGE_SIZE)).convert('RGB')
    image = np.asarray(image)
    image = np.reshape(image, (128, 128, 3))
    print(image.shape)
    # Try with one image. Reshape tries to put it in the shape given.
    # Make list of arrays into one big array
    final.append(np.asarray(image))
    count + +
    print(count)

print("DONE WITH ADDING")
print(type(final))
final = np.reshape(final, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
print("DONE RESHAPING")
print(type(final))
final = final / 127.5 - 1
print(final.shape)
print(final.size)
print(len(final))


np.save('Final.npy', final)


In [ ]:
# Convert catagory of Arch Data into Numbers
cat_list = []
for img_path in all_img_paths:
    for key in catagories.keys():
        if (img_path.find(key) != -1):
            cat_list.append(catagories[key])
print(cat_list)
print(len(cat_list))

In [ ]:
# SWITCH TO y_raw
# One hot encodes the catagory matrix
y_raw = np.asarray(cat_list)
num_classes = len(set(y_raw))
ytry = keras.utils.to_categorical(y_raw, num_classes)
print(len(ytry))
print(ytry)
np.save('YSave', ytry)

In [ ]:
# Run to split test data
x_raw, X_test, y_try, y_test = train_test_split(
    x_raw, ytry, test_size=0.20, shuffle=True)

In [ ]:
# Save all current data
np.save('final.npy', x_raw)
print('finished')
np.save('y_try.npy', y_try)
print('finished')
np.save('X_test.npy', X_test)
print('finished')
np.save('y_test.npy', y_test)
print('finished')

In [ ]:
# Use if just starting
x_raw = np.load('final.npy')

In [ ]:
# Use if already running Final from before
x_raw = final

In [ ]:
y_try = np.load("y_try.npy")

In [ ]:
# Set batchsize and epochs
batch_size = 32
epochs = 3

In [ ]:
# compile and run model
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
hist = model.fit(x_raw, y_try, batch_size=batch_size,
                 epochs=epochs, validation_split=0.1, shuffle=True)
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
# Evaluate on test data
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
# saves model
model.save("testmodel")
print("finished")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x_raw, y_train, test_size=0.20, shuffle=True)

In [ ]:
print(hist.history)
print(hist.history.keys())
# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# undo the onehot encoding
arg = (argmax((y_test), axis=1))
y_test_cat = []
for num in arg:
    y_test_cat.append(num)

In [ ]:
# Calls the model on raw test images and ranks which prediction was correct. 1 = first, 5 = fifth etc.

# Take in Y test
total = []
count = 0
pre1 = []
# Take in raw images
code = 0
i = 0
pre = []
while i < 25:
    pre.append(i)
    i++
for key in catagories.keys():
    for value in pre:
        if catagories[key] == value:
            pre1.append(key)
for image in x_test:
    print(count)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    yhat = model.predict(image)
    yhat1 = yhat * 1000
    predictzip = zip(yhat[0], pre1)
    predictset = set(predictzip)
    sortedpre = (sorted(predictset))
    top5 = sortedpre[-5:]
    val = y_test_cat[count]
    guess = 6
    for key in catagories.keys():
        if catagories[key] == val:
            for tup in top5:
                guess--
                if tup[1] == key:
                    total.append(guess)
    count += 1

print(total)

In [ ]:
# print percentage of model accuracy depending on how many different predicitons you consider.
print(len(total))
lentotal = len(total)
fifth = total.count(5)
print(fifth)
fourth = total.count(4)
print(fourth)
third = total.count(3)
print(third)
sec = total.count(2)
print(sec)
one = total.count(1)
print(one)
top5count = (one + sec + third + fourth+fifth)/lentotal
print("top5count is equal to:" + top5count)
top3count = (one + sec + third)/lentotal
print("top3count is equal to:" + top3count)
print("top3count is equal to:" + (one/lentotal))

In [ ]:
# load the data
x_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')
x_raw = np.load("final.npy")
y_try = np.load("y_try.npy")

In [ ]:

y_pred_ohe = model.predict(x_test)  # shape=(n_samples, 12)
# only necessary if output has one-hot-encoding, shape=(n_samples)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)

confusion_matrix = metrics.confusion_matrix(
    y_true=y_test_cat, y_pred=y_pred_labels)  # shape=(12, 12)

In [ ]:
print(confusion_matrix)

In [ ]:

pl.matshow(confusion_matrix)
pl.show()


In [ ]:
# This does work but you have to have already run different cells. I do not want to have to load up final.py again. 6gb of data
print(classification_report(y_test_cat, y_pred_labels))